# PySpark Installation

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=c08ac70d68f3937b7c7c73a27facf30666362af38560bdc6b7a9468e2ef3608b
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Import Necessary Libraries

In [2]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from google.cloud import bigquery

# Data Preprocessing
## Create The YouTube Trending DataFrame

In [3]:
# Create a Spark session
spark = SparkSession.builder.appName("Chess Data Analysis").getOrCreate()

# Load the CSV files, adding a new column to identify the country of origin
ca_df = spark.read.csv('/content/drive/MyDrive/CSV files/CAvideos.csv', header=True, inferSchema=True, escape='"').withColumn("country", lit("CA"))
gb_df = spark.read.csv('/content/drive/MyDrive/CSV files/GBvideos.csv', header=True, inferSchema=True, escape='"').withColumn("country", lit("GB"))
us_df = spark.read.csv('/content/drive/MyDrive/CSV files/USvideos.csv', header=True, inferSchema=True, escape='"').withColumn("country", lit("US"))

# Union the DataFrames
df = ca_df.union(gb_df).union(us_df)

# Show the combined DataFrame
df.show()

+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
|   video_id|trending_date|                 title|       channel_title|category_id|        publish_time|                          tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|country|
+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
|n1WpP7iowLc|     17.14.11|  Eminem - Walk On ...|          EminemVEVO|         10|2017-11-10T17:00:...|          "Eminem"|"Walk"|"...|17158579| 787

In [4]:
num_of_rows = df.count()
print(f"Number of rows: {num_of_rows}")

Number of rows: 136992


## Analyze The Data Distribution For Each Column

In [5]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- country: string (nullable = false)



## Data Cleaning

After reviewing the schema, we observed datatype issues in the columns for each country. Specifically:

* 'category_id', 'views', 'likes', 'dislikes', and 'comment_count' should be **integers**.
* 'comments_disabled', 'ratings_disabled', and 'video_error_or_removed' should be **booleans**.

The incorrect assignment of schema can be caused by the presence of special characters within each column, hence the string datatype. This discrepancy indicates a need for data cleaning. First, let's make sure that the column, 'category_id', 'views', 'likes', 'dislikes', and 'comment_count' contain non-integer values.

### Filtering

In [6]:
# Filter to only show non-integer values for the "category_id" column.
df.filter(col("category_id").rlike("^[^0-9]+$")).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------+-------------+--------------+-----------------+----------------+----------------------+-----------------+-------+
|            video_id|       trending_date|               title|       channel_title|         category_id|        publish_time|                tags|        views|       likes|      dislikes|comment_count|thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|      description|country|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+--------------+-------------+--------------+-----------------+----------------+----------------------+-----------------+-------+
|\nCook with confi...|             recipes|              videos| and restaurant g...| di

In [7]:
# Filter to only show non-integer values for the "views" column.
df.filter(col("views").rlike("^[^0-9]+$")).show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|            video_id|       trending_date|               title|channel_title|         category_id|        publish_time|                tags|               views|               likes|            dislikes|       comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|      description|country|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|\nCast

In [8]:
# Filter to only show non-integer values for the "likes" column.
df.filter(col("likes").rlike("^[^0-9]+$")).show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|            video_id|       trending_date|               title|channel_title|         category_id|        publish_time|                tags|               views|               likes|            dislikes|       comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|      description|country|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|\nCast

In [9]:
# Filter to only show non-integer values for the "dislikes" column.
df.filter(col("dislikes").rlike("^[^0-9]+$")).show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|            video_id|       trending_date|               title|channel_title|         category_id|        publish_time|                tags|               views|               likes|            dislikes|       comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|      description|country|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|\nCast

In [10]:
# Filter to only show non-integer values for the "comment_count" column.
df.filter(col("comment_count").rlike("^[^0-9]+$")).show()

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|            video_id|       trending_date|               title|channel_title|         category_id|        publish_time|                tags|               views|               likes|            dislikes|       comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|      description|country|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+----------------+----------------------+-----------------+-------+
|\nCast

We've confirmed that the columns do need to undergo **conversion**.

In [11]:
def clean_to_integer(df):
    return df.withColumn("category_id", regexp_replace(col("category_id"), "[^0-9]", "").cast(IntegerType())) \
             .withColumn("likes", regexp_replace(col("likes"), "[^0-9]", "").cast(IntegerType())) \
             .withColumn("dislikes", regexp_replace(col("dislikes"), "[^0-9]", "").cast(IntegerType())) \
             .withColumn("views", regexp_replace(col("views"), "[^0-9]", "").cast(IntegerType())) \
             .withColumn("comment_count", regexp_replace(col("comment_count"), "[^0-9]", "").cast(IntegerType()))

df = clean_to_integer(df)

def clean_and_cast_to_boolean(df):
    return df.withColumn("comments_disabled",
                         when(col("comments_disabled").isin('true', 'True', '1'), True)
                         .when(col("comments_disabled").isin('false', 'False', '0'), False)
                         .otherwise(None).cast(BooleanType())) \
             .withColumn("ratings_disabled",
                         when(col("ratings_disabled").isin('true', 'True', '1'), True)
                         .when(col("ratings_disabled").isin('false', 'False', '0'), False)
                         .otherwise(None).cast(BooleanType())) \
             .withColumn("video_error_or_removed",
                         when(col("video_error_or_removed").isin('true', 'True', '1'), True)
                         .when(col("video_error_or_removed").isin('false', 'False', '0'), False)
                         .otherwise(None).cast(BooleanType()))

df = clean_and_cast_to_boolean(df)

df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- video_error_or_removed: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- country: string (nullable = false)



The **datatype** of each column is now correct.

In [12]:
df.show()

+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
|   video_id|trending_date|                 title|       channel_title|category_id|        publish_time|                          tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|country|
+-----------+-------------+----------------------+--------------------+-----------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+
|n1WpP7iowLc|     17.14.11|  Eminem - Walk On ...|          EminemVEVO|         10|2017-11-10T17:00:...|          "Eminem"|"Walk"|"...|17158579| 787

### Joining

The column "category_id" contains **integer** values representing category IDs. Through a separate discussion on Kaggle.com, it is found that the category IDs correspond to these categories:
- 1, 'Film & Animation'
- 2, 'Autos & Vehicle'
- 10, 'Music'
- 15, 'Pets & Animals'
- 17, 'Sports'
- 19, 'Travel & Events'
- 20, 'Gaming'
- 22, 'People & Blogs'
- 23, 'Comedy'
- 24, 'Entertainment'
- 25, 'News & Politics'
- 26, 'Howto & Style'
- 27, 'Education'
- 28, 'Science & Technology'
- 30, 'Movies'
- 43, 'Shows'
- 29, 'Nonprofits & Activism'

Through this, we are able to replace each integer value with its corresponding categories.

In [13]:
category_data = [(1,'Film & Animation'),(2,'Autos & Vehicle'),(10,'Music'),(15,'Pets & Animals'),
                 (17,'Sports'),(19,'Travel & Events'),(20,'Gaming'),(22,'People & Blogs'),(23,'Comedy'),
                 (24,'Entertainment'),(25,'News & Politics'),(26,'Howto & Style'),(27,'Education'),
                 (28,'Science & Technology'),(30,'Movies'),(43,'Shows'),(29,'Nonprofits & Activism')]

category_df = spark.createDataFrame(category_data, ['category_id', 'category_name'])

df = df.join(category_df, on='category_id', how='left')

df.show()

+-----------+-----------+-------------+----------------------+------------------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+--------------------+
|category_id|   video_id|trending_date|                 title|     channel_title|        publish_time|                          tags|   views|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|country|       category_name|
+-----------+-----------+-------------+----------------------+------------------+--------------------+------------------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+--------------------+
|         26|U3xLOo-CNwo|     17.14.11|  12 Weird Ways To ...|       Troom Troom|2017-11-12

### Handling Missing Values

In [14]:
# Count the amount of missing values in each column.
null_count = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_count.show()

+-----------+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+-------+-------------+
|category_id|video_id|trending_date|title|channel_title|publish_time| tags|views|likes|dislikes|comment_count|thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|description|country|category_name|
+-----------+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+-----------------+----------------+----------------------+-----------+-------+-------------+
|      16219|       0|        13420|13841|        14091|       14905|15843|16243|16246|   16246|        16245|         16017|            16246|           16246|                 16246|      18666|      0|        16245|
+-----------+--------+-------------+-----+-------------+------------+-----+-----+-----+--------+-------------+--------------+---

In [15]:
# Compare to the current number of rows.
num_of_rows = df.count()
print(f"Number of rows: {num_of_rows}")

Number of rows: 136992


In [16]:
# Drop the missing values
df_cleaned = df.dropna()
df_cleaned.show()

+-----------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+--------------------+
|category_id|   video_id|trending_date|               title|       channel_title|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|country|       category_name|
+-----------+-----------+-------------+--------------------+--------------------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-------+--------------------+
|         26|U3xLOo-CNwo|     17.14.11|12 Weird Ways To ...|         Troom Troom|2017-11-12T15:00:...|"sneak food"|"how...|3153

In [17]:
# Check for the number of rows after dropping the missing values.
num_of_rows = df_cleaned.count()
print(f"Number of rows: {num_of_rows}")

Number of rows: 118268


From this, we can infer that the rows with missing values generally have a high proportion of their columns with missing values. Hence, we are able to drop the rows with missing values, as they do not provide much data for further analysis.

## DataFrame Description
This dataset includes several months (and counting) of data on daily trending YouTube videos. Data is included for the US, GB, and CA regions (USA, Great Britain, and Canada respectively), with up to 200 listed trending videos per day.
### Columns:
1. **Video ID** (string): A unique identifier for the video, assigned by YouTube. It’s used to identify each video on the platform.
2. **Trending Date** (string): The date the video appeared on the trending list. This is typically formatted as 'YY.DD.MM'.
3. **Title** (string): The title of the video, as provided by the content creator. It gives a brief description of the video content.
4. **Channel Title** (string): The name of the channel that uploaded the video. This represents the content creator or the entity behind the video.
5. **Category ID** (string): A numeric ID that represents the category the video belongs to. It corresponds to predefined YouTube categories.
6. **Publish Time** (string): The exact time the video was published on YouTube, typically in ISO 8601 format (YYYY-MM-DDTHH:MM:SSZ).
7. **Tags** (string): Keywords or phrases used to describe the video. These are provided by the content creator to help with searchability and categorization.
8. **Views** (integer): The total number of views the video has received as of the trending date.
9. **Likes** (integer): The total number of "likes" the video has received from viewers.
10. **Dislikes** (integer): The total number of "dislikes" the video has received from viewers.
11. **Comment Count** (integer): The total number of comments posted on the video.
12. **Thumbnail Link** (string): The URL link to the thumbnail image for the video. This is a preview image used to represent the video on the platform.
13. **Comments Disabled** (boolean): A boolean value (True or False) indicating whether comments are disabled for the video.
14. **Ratings Disabled** (boolean): A boolean value (True or False) indicating whether ratings (likes/dislikes) are disabled for the video.
15. **Video Error or Removed** (boolean): A boolean value (True or False) indicating whether the video encountered an error or was removed by YouTube or the uploader.
16. **Description** (string): The description of the video, provided by the content creator. It offers more details about the video content, including links or additional information.
17. **Country** (string): The country where the video was trending. It indicates the region-specific trending list the video appeared on.


## Data Analysis

### 1. Average Views by Category

In [45]:
df_cleaned.createOrReplaceTempView("df_cleaned")
# Using SQL for aggregating engagement by category
spark.sql("""
SELECT category_name, AVG(views) AS avg_views
FROM df_cleaned
GROUP BY category_name
ORDER BY avg_views DESC
""").show(truncate=False)

+---------------------+------------------+
|category_name        |avg_views         |
+---------------------+------------------+
|Music                |9423559.893274484 |
|Nonprofits & Activism|3383961.354037267 |
|Movies               |2853415.0         |
|Film & Animation     |2691624.977523202 |
|Entertainment        |1973317.810990867 |
|Sports               |1592139.4128509234|
|Science & Technology |1527881.1904522614|
|Comedy               |1419650.9505690695|
|People & Blogs       |1373989.1819885476|
|Gaming               |1300494.7303543913|
|Autos & Vehicle      |1070253.4902411022|
|Pets & Animals       |850045.5057915058 |
|Howto & Style        |848065.007941432  |
|Shows                |754397.2587064677 |
|Education            |698450.4933463162 |
|Travel & Events      |691532.1874285714 |
|News & Politics      |592146.2959279667 |
+---------------------+------------------+



### 2. Total Engagement Rate

In [19]:
spark.sql("""
SELECT title, channel_title, country, category_name,
    ((COALESCE(likes,0)+COALESCE(dislikes,0)+COALESCE(comment_count,0))/views)*100 AS engagement_rate
FROM df_cleaned
WHERE views > 0
ORDER BY engagement_rate DESC
""").show(truncate=False)

+----------------------------------------------------------------------------------------------+-------------------+-------+----------------+------------------+
|title                                                                                         |channel_title      |country|category_name   |engagement_rate   |
+----------------------------------------------------------------------------------------------+-------------------+-------+----------------+------------------+
|BlocBoy JB & Drake Look Alive Prod By: Tay Keith (Official Music Video) Shot By: @Fredrivk_Ali|BlocBoy JB         |CA     |Music           |50.87536744621628 |
|PADMAN Official Trailer | Akshay Kumar | Sonam Kapoor | Radhika Apte | 26th Jan 2018          |Sony Pictures India|CA     |Entertainment   |45.148817351112704|
|GUCCI MADE ME PINK CUSTOM LUGGAGE! + HANDBAG GIVEAWAY!                                        |jeffreestar        |CA     |Howto & Style   |43.90593407351642 |
|Why I'm Deleting SkyDoesMinecraft

### 3. Time to Trend (in Days)

In [20]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

df_cleaned = df_cleaned.withColumn('trending_date',
                                   F.to_date(
                                       F.concat(
                                          F.lit('20'),
                                           F.substring('trending_date', 1, 2),
                                           F.lit('-'),
                                           F.substring('trending_date', 7, 2),
                                           F.lit('-'),
                                           F.substring('trending_date', 4, 2)
                                      ),'yyyy-MM-dd'
                                    )
                                  )

# Correctly create publish_time from publish_time column
df_cleaned = df_cleaned.withColumn('publish_time', to_date('publish_time', 'yyyy-MM-dd'))

# Create a temporary view
df_cleaned.createOrReplaceTempView("df_cleaned")

# Use the correct column name in SQL
spark.sql("""
SELECT title, channel_title, category_name, country,
  DATEDIFF(trending_date, publish_time) AS days_to_trend
FROM df_cleaned
WHERE trending_date IS NOT NULL AND publish_time IS NOT NULL
ORDER BY days_to_trend DESC
""").show(truncate=False)

+------------------------------------+-------------+-------------+-------+-------------+
|title                               |channel_title|category_name|country|days_to_trend|
+------------------------------------+-------------+-------------+-------+-------------+
|Budweiser - Original Whazzup? ad    |dannotv      |Entertainment|US     |4215         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3990         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3989         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3988         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3987         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3986         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3985         |
|Jeffrey Lewis The Legend of The Fall|Dav hum      |Music        |GB     |3984         |
|Jeffrey Lewis The Le

In [21]:
spark.sql("""
SELECT title, channel_title, category_name, country,
  DATEDIFF(trending_date, publish_time) AS days_to_trend
FROM df_cleaned
WHERE trending_date IS NOT NULL AND publish_time IS NOT NULL
ORDER BY days_to_trend
""").show(truncate=False)

+--------------------------------------------------------------------------------------------+--------------------+---------------------+-------+-------------+
|title                                                                                       |channel_title       |category_name        |country|days_to_trend|
+--------------------------------------------------------------------------------------------+--------------------+---------------------+-------+-------------+
|Tyron Woodley Says Things Got Real With Conor McGregor | The Hollywood Beatdown | TMZ Sports|TMZSports           |Sports               |GB     |0            |
|FIFTY SHADES OF GREY LOOKBOOK!                                                              |Amber Scholl        |Howto & Style        |US     |0            |
|Using Failure as Design                                                                     |Peter Brown         |Howto & Style        |CA     |0            |
|Kobe Bryant Jersey Retirement Press Con

### 4. Likes-to-Dislikes Ratio

In [22]:
spark.sql("""
SELECT category_name, AVG(likes/dislikes) AS likes_to_dislikes_ratio
FROM df_cleaned
GROUP BY category_name
ORDER BY likes_to_dislikes_ratio DESC
""").show(truncate=False)

+---------------------+-----------------------+
|category_name        |likes_to_dislikes_ratio|
+---------------------+-----------------------+
|Pets & Animals       |66.51641629313197      |
|Music                |58.58219104962315      |
|Howto & Style        |54.0937639466431       |
|People & Blogs       |53.92206833317354      |
|Comedy               |52.73102765893661      |
|Gaming               |50.38198870463376      |
|Education            |49.879980756384484     |
|Nonprofits & Activism|46.12641508937074      |
|Travel & Events      |41.63250289337735      |
|Film & Animation     |41.351937508026026     |
|Entertainment        |41.27796368739464      |
|Science & Technology |39.88332262597649      |
|Autos & Vehicle      |38.33470295465094      |
|Sports               |31.840352976072946     |
|Shows                |30.085099892906623     |
|Movies               |25.141341335119517     |
|News & Politics      |17.383123710232315     |
+---------------------+-----------------

### 5. Comment Count

In [23]:
spark.sql("""
SELECT category_name, AVG(comment_count) AS avg_comment_count
FROM df_cleaned
GROUP BY category_name
ORDER BY avg_comment_count DESC
""").show(truncate=False)

+---------------------+------------------+
|category_name        |avg_comment_count |
+---------------------+------------------+
|Nonprofits & Activism|96281.3850931677  |
|Music                |19739.67818411097 |
|Gaming               |7825.992039034412 |
|Entertainment        |6805.789472038034 |
|Comedy               |6526.466413746932 |
|Film & Animation     |6313.71258700696  |
|People & Blogs       |5900.555752212389 |
|Science & Technology |5546.90351758794  |
|Howto & Style        |4818.714232535054 |
|Sports               |4308.493019066206 |
|Education            |3258.4703018500486|
|Pets & Animals       |3248.0645339216767|
|News & Politics      |3239.742776508534 |
|Autos & Vehicle      |2644.6222732491387|
|Movies               |2329.8333333333335|
|Travel & Events      |1852.4651428571428|
|Shows                |1260.2835820895523|
+---------------------+------------------+



### 6. View Velocity

In [24]:
spark.sql("""
SELECT
    title,
    channel_title,
    category_name,
    country,
    DATEDIFF(trending_date, publish_time) AS days_to_trend,
    CASE
        WHEN DATEDIFF(trending_date, publish_time) > 0 THEN views / DATEDIFF(trending_date, publish_time)
        ELSE NULL
    END AS view_velocity
FROM df_cleaned
WHERE trending_date IS NOT NULL AND publish_time IS NOT NULL
GROUP BY title, channel_title, category_name, country, days_to_trend, view_velocity
ORDER BY view_velocity DESC
""").show(truncate=False)

+-------------------------------------------------------+---------------------------+-------------+-------+-------------+-------------+
|title                                                  |channel_title              |category_name|country|days_to_trend|view_velocity|
+-------------------------------------------------------+---------------------------+-------------+-------+-------------+-------------+
|BTS (방탄소년단) 'FAKE LOVE' Official MV               |ibighit                    |Music        |CA     |1            |3.9349927E7  |
|BTS (방탄소년단) 'FAKE LOVE' Official MV               |ibighit                    |Music        |GB     |1            |3.9349927E7  |
|BTS (방탄소년단) 'FAKE LOVE' Official MV               |ibighit                    |Music        |US     |1            |3.9349927E7  |
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel Entertainment       |Entertainment|CA     |1            |3.7736281E7  |
|Marvel Studios' Avengers: Infinity War Official Trailer|Marvel

### 7. Channel/Creator Performance

In [25]:
spark.sql("""
SELECT channel_title,
  AVG(views) AS avg_views,
  AVG(likes) AS avg_likes,
  AVG(dislikes) AS avg_dislikes,
  AVG(comment_count) AS avg_comment_count
FROM df_cleaned
GROUP BY channel_title
""").show(truncate=False)

+----------------------------------------------+------------------+------------------+------------------+------------------+
|channel_title                                 |avg_views         |avg_likes         |avg_dislikes      |avg_comment_count |
+----------------------------------------------+------------------+------------------+------------------+------------------+
|Kai Bent-Lee                                  |33741.5           |1699.0            |37.0              |167.0             |
|The New York Times                            |446517.2          |5813.627272727273 |826.8909090909091 |1745.1363636363637|
|Suspicious0bservers                           |34547.0           |1856.0            |22.0              |303.0             |
|SPORTSNET                                     |163962.0693069307 |916.1089108910891 |44.24752475247525 |370.3168316831683 |
|优优独播剧场——YoYo Television Series Exclusive|81231.16326530612 |410.3265306122449 |21.653061224489797|155.57142857142858|
|Hi Sp

### 8. Category Performance

In [26]:
spark.sql("""
SELECT category_name,
  AVG(views) AS avg_views,
  AVG(likes) AS avg_likes,
  AVG(dislikes) AS avg_dislikes,
  AVG(comment_count) AS avg_comment_count
FROM df_cleaned
GROUP BY category_name
""").show(truncate=False)

+---------------------+------------------+------------------+------------------+------------------+
|category_name        |avg_views         |avg_likes         |avg_dislikes      |avg_comment_count |
+---------------------+------------------+------------------+------------------+------------------+
|Shows                |754397.2587064677 |12480.796019900497|752.9353233830846 |1260.2835820895523|
|Education            |698450.4933463162 |28468.749756572543|887.3885102239533 |3258.4703018500486|
|Gaming               |1300494.7303543913|48764.97817154597 |3563.598870056497 |7825.992039034412 |
|Entertainment        |1973317.810990867 |50964.4262166896  |3908.400287751783 |6805.789472038034 |
|Travel & Events      |691532.1874285714 |10714.622857142856|557.3451428571428 |1852.4651428571428|
|Science & Technology |1527881.1904522614|37718.82311557789 |1938.972864321608 |5546.90351758794  |
|Sports               |1592139.4128509234|34834.9450532953  |2737.751538807987 |4308.493019066206 |


### 9. Cross-Country Reach

In [27]:
spark.sql("""
SELECT country, AVG(views) AS avg_views, AVG(likes) AS avg_likes, AVG(dislikes) AS avg_dislikes, AVG(comment_count) AS avg_comment_count
FROM df_cleaned
GROUP BY country
ORDER BY avg_views DESC
""").show(truncate=False)

+-------+------------------+------------------+------------------+-----------------+
|country|avg_views         |avg_likes         |avg_dislikes      |avg_comment_count|
+-------+------------------+------------------+------------------+-----------------+
|GB     |5970072.806913116 |135676.84043441937|7118.518849206349 |12709.73950501253|
|US     |2370838.716833007 |74616.77275316378 |3508.1219693404987|8287.396196042497|
|CA     |1169234.0054313503|40596.940154098775|2058.693267651888 |5159.71628141973 |
+-------+------------------+------------------+------------------+-----------------+



### 10. Trending Retention Rate

In [44]:
spark.sql("""
-- Counts how many times each video has appeared in the trending list for each category
WITH video_trend_count AS (
    SELECT category_name, video_id, COUNT(*) AS trending_count
    FROM df_cleaned
    GROUP BY category_name, video_id
)

-- Identifies videos that trended more than once for each category (retained videos)
, retained_videos AS (
    SELECT category_name, COUNT(DISTINCT video_id) AS retained_videos
    FROM video_trend_count
    WHERE trending_count > 1
    GROUP BY category_name
)

-- Calculates the total number of videos per category
, total_videos AS (
    SELECT category_name, COUNT(DISTINCT video_id) AS total_videos
    FROM df_cleaned
    GROUP BY category_name
)

-- Calculates the retention rate for each category
SELECT
    t.category_name,
    COALESCE(r.retained_videos, 0) * 100.0 / t.total_videos AS retention_rate
FROM total_videos t
LEFT JOIN retained_videos r
ON t.category_name = r.category_name
ORDER BY retention_rate DESC

""").show(n=100)

+--------------------+------------------+
|       category_name|    retention_rate|
+--------------------+------------------+
|              Movies|100.00000000000000|
|               Music| 78.70257037943696|
|     Travel & Events| 74.78991596638655|
|      Pets & Animals| 69.63190184049080|
|Science & Technology| 65.80717488789238|
|              Comedy| 64.28258007898201|
|       Howto & Style| 59.43661971830986|
|           Education| 58.78467635402906|
|              Gaming| 56.05427974947808|
|    Film & Animation| 54.92451473759885|
|     Autos & Vehicle| 50.16393442622951|
|      People & Blogs| 49.46043165467626|
|       Entertainment| 48.62047177759056|
|Nonprofits & Acti...| 45.16129032258065|
|              Sports| 44.59869848156182|
|     News & Politics| 38.30326052548275|
|               Shows| 14.15929203539823|
+--------------------+------------------+

